In [1]:
data_path = 'D:\\NLP\\data\\SMS_Spam\\'
file_name = 'spam.csv'
import pandas as pd

df = pd.read_csv(data_path+file_name, encoding="ISO-8859-1")
df.rename(columns={'v1': 'target', 'v2': 'text'}, inplace=True)

df = df[['text', 'target']]

text_col = 'text'
target_col = 'target'

# X = df[text_col].values
# y = df[target_col].values

X = df[text_col]
y = df[target_col]

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=0)

#df = df.dropna(subset=['text', 'target'], inplace=True) #TBC
print(f'X_train:{type(X_train)}')

df.head()

X_train:<class 'pandas.core.series.Series'>


,text,target
0,"Go until jurong point, crazy.. Available only ...",ham
1,Ok lar... Joking wif u oni...,ham
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam
3,U dun say so early hor... U c already then say...,ham
4,"Nah I don't think he goes to usf, he lives aro...",ham


In [2]:
import nbimporter

from Common_Functions import *


from PP_Pipeline_Functions import *
%store -r feature_create_dict
%store -r pp_dict
%store -r model
%store -r params
%store -r feature_drop_dict
%store -r feature_discretisation_dict
%store -r feature_transformer_dict
%store -r feature_outlier_dict
%store -r feature_lagfreqwindow_dict
%store -r feature_datetime_dict
%store -r feature_datetime_list
%store -r feature_lagfreqwindow_list
%store -r feature_selection_dict
%store -r imputer_dict
%store -r imputaion_data_type_dict
%store -r encoder_dict
%store -r simple_encoder_dict
%store -r scaler_dict
%store -r sample_type_dict
%store -r reduction_type_dict

%store -r DropDupRows_fs
%store -r DropNullRows_fs
%store -r DropDupRows_ft
%store -r DropNullRows_ft

%store -r GroupFeatures_ft
%store -r GroupFeatures_fs


from Models_Functions import *
%store -r models_fname_listR
%store -r models_sname_listR
%store -r models_fdictR
%store -r models_sdictR
%store -r params_pipeline_fdictR
%store -r params_pipeline_sdictR

%store -r models_fname_listC
%store -r models_sname_listC
%store -r models_fdictC
%store -r models_sdictC
%store -r params_pipeline_fdictC
%store -r params_pipeline_sdictC

%store -r tt_cv_pt_dict

%store -r keras_metrics_flistR
%store -r keras_metrics_slistR
%store -r keras_metrics_flistC
%store -r keras_metrics_slistC

from NN_Functions import *
from NLP_Functions import *
#from TimeSeries_Functions import *

In [ ]:
read_file

In [3]:
#1.6 / Oct01,2022 / moved all functions to respective functions
'''
Conut_vect /tf-idf/Glove_Embedding to be added as check box of pipeline thr column transformer
Text feature/clean/Aug/scaling/selection to be added as check box of pipeline
folder creation as per traget selection / folderpath text i/p to be added
pp graph : traget distribution / text word cloud to be added

ML_Models:
Train_test/corss_val tt_cv_pt latest function TBU

'''
import dash
from dash import dcc, html, dash_table
import dash_bootstrap_components as dbc

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from dash.dependencies import State, Input, Output
from dash.exceptions import PreventUpdate
import dash_daq as daq

import pandas as pd
import numpy as np
from datetime import date

import os
import pickle

#external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

#app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
#path
project = 'spam_class'

# data_path = 'D:\\NLP\\data\\'
# pred_path = 'D:\\NLP\\pred\\'
# #pred_consolidated_path = 'D:\\NLP\\pred\\consolidated\\'
# reports_path = 'D:\\NLP\\reports\\'
# model_path = 'D:\\NLP\\models\\'
# NN_model_path = 'D:\\NLP\\models\\NN_models'
# plot_path = 'D:\\NLP\\plots\\'

#glove details
glove_path = 'D:\\NLP\\glove_word_embedding\\'
file50d = 'glove.6B.50d.txt'
file100d = 'glove.6B.100d.txt'
file200d = 'glove.6B.200d.txt'
glove_file = file50d 

#tab pp_mpdel variables
STOPWORDS_list = ['a', 'an', 'the']

#tab ml_model variables
score_error_list = ['test_recall', 'test_f1', 'loss_log', 'auc_roc_score', 'auc_prc_score', 'TPR', 'FPR', 'TNR', 'FNR', 'ACC', 'PPV', 'NPV']
feat_impot_list = ['VarianceThreshold', 'SelectKBest', 'SelectPercentile', 'SelectFromModel', 'RFE', 'RFECV', 'SelectFpr', 'SelectFdr', 'SelectFwe']

#tab nn_model variables
#nn_model_list_lrng = ['lrng_embd_LSTM', 'lrng_embd_NN', 'lrng_embd_ShlowNN']
#nn_model_list_trnd = ['trnd_embd_LSTM1', 'trnd_embd_CNN1D', 'trnd_embd_LSTM2', 'trnd_embd_GRU1', 'trnd_embd_BidirGRU1', 'trnd_embd_BidirRNN1', 'trnd_embd_BidirCuDNNLSTM']

nn_model_list = ['lrng_embd_LSTM', 'lrng_embd_NN', 'lrng_embd_ShlowNN', 'trnd_embd_LSTM1', 'trnd_embd_CNN1D',
        'trnd_embd_LSTM2', 'trnd_embd_GRU1', 'trnd_embd_BidirGRU1', 'trnd_embd_BidirRNN1', 'trnd_embd_BidirCuDNNLSTM']
nn_score_error_list = ['mean_absolute_error', 'mean_absolute_percentage_error', 'mean_squared_error', 'mean_squared_logarithmic_error', 'mean']
nn_score_error_list1 = list(nn_score_error_list) # copy list , it will not impact to orignal list
nn_score_error_list1.remove('mean') # NN_model does not support mean / TBC


app.layout = html.Div([
    #html.H1('Dash Tabs component demo'),
    dcc.Tabs(id="NLP_Classification", value='NLP Classification', children=[
        dcc.Tab(label='PreProcess', value='Text_PP', children=[
            html.Br(),
            dbc.Row([
                dbc.Col(dcc.Upload(id='upload-data', children=html.Div(['Drag and Drop or ', html.A('Select Files')]))
                       ),
                
                dbc.Col(html.Div(["Select ngrams max and min range",
                        dcc.RangeSlider(
                        id='text_ngram_slider',
                        min=0,
                        max=5,
                        step=1,
                        value=[1, 2], tooltip = { 'always_visible': True })])
                        ),
            ]), #r1
            html.Br(),

            dbc.Row([
                dbc.Col(dcc.Dropdown(id="pp_text_dd",
                    multi=False,
                    placeholder="Select Text data column",
                    clearable=False,  
                    searchable=False),
                    ), #r2c1
                    
                dbc.Col(dcc.Dropdown(id="pp_target_dd",
                    multi=False,
                    placeholder="Select target column",
                    clearable=False,  
                    searchable=False),
                    ), #r2c2
                
                dbc.Col(daq.ToggleSwitch(id='pp_text_feat_chk', label='Text Features', value=False)),
                dbc.Col(daq.ToggleSwitch(id='pp_text_clean_chk', label='Text Clean', value=False)),
                dbc.Col(daq.ToggleSwitch(id='pp_text_aug_chk', label='Text Augmentation', value=False)),
                dbc.Col(daq.ToggleSwitch(id='pp_data_scale_chk', label='Feature Scaling', value=False)),
                dbc.Col(daq.ToggleSwitch(id='pp_feat_select_chk', label='Imp Feature Selection', value=False)),
                
            ]), #r2
            html.Br(),
            
            dbc.Row([
                dbc.Col(dcc.RadioItems(
                id='pp_text_encoding_radio', value='Count_vect',
                options=[{'label': x, 'value': x} for x in [ 'Count_vect', 'TfIdf_vect', 'Glove_Embedding']],
                labelStyle={'width': '100px', 'display': 'inline-block'}),
                       ), #r4c6
                
                dbc.Col(dcc.RadioItems(
                id='pp_data_type_radio', value='Dense',
                options=[{'label': x, 'value': x} for x in [ 'Dense', 'Sparse']],
                labelStyle={'width': '100px', 'display': 'inline-block'}),
                      ), #r4c6
                
            ]), #r3
            html.Br(),
            html.Div([
                dbc.Row([
                    dbc.Col(dcc.Input(id="pp_min_df", type="number", placeholder="enter min df", min=0.0, max=1.0, debounce=True,
                              style={'width': '200px', 'margin-left':'10px', 'display': 'inline-block'})
                           ),

                    dbc.Col(dcc.Input(id="pp_max_df", type="number", placeholder="enter max df", min=0.0, max=1.0, debounce=True,
                              style={'width': '200px', 'margin-left':'10px', 'display': 'inline-block'})
                           ),

                    dbc.Col(dcc.Input(id="pp_max_feat", type="number", placeholder="enter max features", debounce=True,
                              style={'width': '200px', 'margin-left':'10px', 'display': 'inline-block'})
                           ),

                    dbc.Col(dcc.Dropdown(
                        id="pp_analyzer_dd",
                        options=[{'label':x, 'value':x} for x in ['word', 'char', 'char_wb']],
                        multi=False,
                        placeholder="Select analyzer",
                        clearable=False,  
                        searchable=False,
                        style={'width': '200px', 'margin-left':'10px', 'display': 'inline-block'})
                           ),
                ]), #r4
            ], style= {'display': 'block'}), #html.Div , will be changed by the dropdown callback
            html.Br(),
            

                #4 data button
                html.Button("Preprocess", id="pp_btn"),

                # output dispaly
                html.Div(id="output_data_pp"),

                #tab ts_train end
                html.Br(),
                dcc.Graph(id="Data_PP"),
    
            ]), # # end tab data_pp

        
        
        dcc.Tab(label='ML Models', value='ML_Models', children=[
            html.Div(style={'color': 'blue', 'fontSize': 16, 'textAlign': 'center'}, children=[
            html.Br(),
            dbc.Row(
                dbc.Col(dcc.Dropdown(
                    id="ml_pp_ptrn_dd",
                    multi=False,
                    placeholder="Select PP pattern data",
                    clearable=False,  
                    searchable=False,
                    style={'width': '1000px', 'margin-left':'10px', 'display': 'inline-block'})
                       )
                ),
                html.Br(),
                
            dbc.Row(
                dbc.Col(dcc.Checklist(id='ml_models_chk',
                    #options=[{'label':x, 'value':x} for x in list(models_dict.keys())], 
                    options=[{'label':x, 'value':x} for x in list(models_fdictC.keys())], # Oct01,2022
                    value=['LogisticRegression'],
                    #style={'width': '200px', 'margin-left':'10px', 'display': 'inline-block'}
                    labelStyle={"display": "inline-block"})
                       ),
                ),
                 #score/error dropdown
            dbc.Row([
                dbc.Col(dcc.Dropdown(
                    id="ml_score_dd",
                    options=[{'label':x, 'value':x} for x in score_error_list],
                    multi=False,
                    placeholder="Select score/error Name",
                    clearable=False,  
                    searchable=False,
                    style={'width': '200px', 'margin-left':'10px', 'display': 'inline-block'})
                       ),
                
                dbc.Col(dcc.RadioItems(
                id='pp_training_type_radio', value='TrainTest',
                options=[{'label': x, 'value': x} for x in [ 'TrainTest', 'CrossValid', 'GridSearch']],
                labelStyle={'width': '100px', 'display': 'inline-block'})
                        ), #r4c6
            ]),
                html.Br(),
                
                #4 download data button
                html.Button("train/test ML Models", id="ml_btn"),

                # ts_train output dispaly
                html.Div(id="output_ml_model"),

                 #tab ts_train end
                   ], className='row'),
                   html.Br(),
                   dcc.Graph(id="ML_model"),
        ]), # end tab ML_Models
        dcc.Tab(label='NN Models', value='NN_Models', children=[
            html.Br(),
            dbc.Row(
                dbc.Col(dcc.Dropdown(
                    id="nn_pp_ptrn_dd",
                    multi=False,
                    placeholder="Select PP pattern data",
                    clearable=False,  
                    searchable=False),
                    width={"size": 10},
                    ),
            justify="center",), #r1
            
            html.Br(),
            dbc.Row(
                #dbc.Col(
                dbc.Checklist(id='nn_models_chk',
                    options=[{'label':x, 'value':x} for x in nn_model_list],
                    value=['lrng_embd_NN'],
                    inline=True),
            ), #r2
            
            #html.Br(),
            dbc.Row(
                #dbc.Col(
                dbc.Checklist(id='nn_cols_chk',
                    inline=True,
                    labelStyle={ "display": "inline-block"}),
                 #   ),
            ), #r3
            
            html.Br(),
            dbc.Row([
                dbc.Col(dbc.Input(id="n_epochs", type="number", placeholder="enter n_epochs", debounce=True)
                    ),
                dbc.Col(dbc.Input(id="n_neurons", type="number", placeholder="enter n_neurons", debounce=True)
                    ),
                dbc.Col(dbc.Input(id="n_batch", type="number", placeholder="enter n_batch", debounce=True)
                    ),
                dbc.Col(dbc.Input(id="n_repeats", type="number", placeholder="enter n_repeats", debounce=True)
                    ),
            ]), #r4
            
            html.Br(),
            dbc.Row([ 
                dbc.Col(dcc.Dropdown(
                    id="nn_optimizer_dd",
                    options=[{'label':x, 'value':x} for x in ['Adadelta', 'Adagrad', 'Adam', 'RMSprop', 'SGD']],
                    multi=False,
                    placeholder="Select optimizer",
                    clearable=False,  
                    searchable=False)
                    ),
                
                dbc.Col(dcc.Dropdown(
                    id="nn_score_dd",
                    options=[{'label':x, 'value':x} for x in nn_score_error_list1],
                    multi=False,
                    placeholder="Select score matrix",
                    clearable=False,  
                    searchable=False)
                    ),
                
                dbc.Col(dcc.Dropdown(
                    id="nn_loss_dd",
                    options=[{'label':x, 'value':x} for x in nn_score_error_list1],
                    multi=False,
                    placeholder="Select loss function",
                    clearable=False,  
                    searchable=False)
                    ),
            ]), #r5
            
            html.Br(),
            dbc.Row([
                dbc.Col(html.Div(["Select learning rate",
                    dcc.Slider(
                        id='nn_lr_slider',
                        min=.001,
                        max=0.1,
                        step=0.001,
                        value=0.1,  tooltip = { 'always_visible': True })],)
                    ), 
                
            ]), #r6
            
            dbc.Row(
                dbc.Col(html.Button('train/test NN Models', id='nn_btn'),
                       width={"size": 6, "offset": 3},),
                align="center", justify="center",), #r5
            
                html.Div(id="output_nn_model"),

               #], className='row'), #html.Div(style close
        
               html.Br(),
               dcc.Graph(id="NN_model"),
        ]),
        
        dcc.Tab(label='Stat Models', value='Stat_Models', children=[]),
        
    ]) #Tabs
]) #layout

# Tab: Text_PP
# def read_file(contents, filename):
#     import base64
#     import io
    
#     print('read_file')
#     if contents is not None:
#         content_type, content_string = contents.split(',')
#         global df
        
#         decoded = base64.b64decode(content_string)
#         try:
#             if 'csv' in filename:  #user uploaded a CSV file
#                 #df = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
#                 df = pd.read_csv(io.StringIO(decoded.decode('ISO-8859-1')))
#                 print('csv:', df.shape)
#             elif 'xls' in filename: #user uploaded a xls file
#                 df = pd.read_excel(io.BytesIO(decoded))
#                 print('xls:', df.shape)
#         except Exception as e:
#             print(e)
#         return df

@app.callback(
      [Output('pp_text_dd', 'options'),
      Output('pp_target_dd', 'options'),
      ],
      Input('upload-data', 'contents'), # chk file contents
      State('upload-data', 'filename'),
      )

def data_pp_dd_text_target1(contents, filename):
    print('callback data_pp_dd_text_target1')
    global df # moved from read_file Oct2,2022
    
    df = read_file(contents, filename)
    print(f'df-shp:{df.shape}, df-cols:{df.columns.tolist()}')
    
    if contents is not None:
        option=[{'label':x, 'value':x} for x in df.columns.tolist()]
        #print('list_of_contents is not None')
        options = [option, option]
        return options
    
    
@app.callback(
      Output(component_id='pp_min_df', component_property='style'),
#      Output(component_id='pp_max_df', component_property='style'),
#      Output(component_id='pp_max_feat', component_property='style'),
#      Output(component_id='pp_analyzer_dd', component_property='style')
   [Input(component_id='pp_text_encoding_radio', component_property='value')])

def show_hide_element_Data_pp2(visibility_state):
    print('callback show_hide_element_Data_pp2')
    if (visibility_state == 'CountVect') or (visibility_state == 'TfIdf'):
        print('if CountVect/tfidf')
        return {'display': 'block'}
    else:
        print('else other')
        return {'display': 'none'}
    
    
@app.callback(
    [
    Output('Data_PP', 'figure'),
    Output("output_data_pp", "children"),
    ],
    [
     Input('pp_text_dd', 'value'),
     Input('pp_target_dd', 'value'),
     Input('pp_text_feat_chk', 'value'), #
     Input('pp_text_clean_chk', 'value'),
     Input('pp_text_aug_chk', 'value'),
     Input('pp_data_scale_chk', 'value'),
        
     Input('pp_text_encoding_radio', 'value'),
     Input('pp_min_df', 'value'),
     Input('pp_max_df', 'value'),
     Input('pp_max_feat', 'value'),
     
     Input('pp_ngram_slider', 'value'),
    
     Input('pp_analyzer_dd', 'value'),
     Input('pp_feat_select_chk', 'value'),
     Input('pp_data_type_radio', 'value'),
     Input("pp_btn", "n_clicks")
    ], prevent_initial_call=True)

def update_Data_pp_final3(pp_text_dd, pp_target_dd, pp_text_feat_chk, pp_text_clean_chk, pp_text_aug_chk, pp_data_scale_chk,
                   pp_text_encoding_radio, pp_min_df, pp_max_df, pp_max_feat, pp_ngram_slider, pp_analyzer_dd,
                   pp_feat_select_chk, pp_data_type_radio, pp_btn):
    
    from sklearn.model_selection import train_test_split
    print('callback update_Data_pp_final3')
    print(f'df-shp:{df.shape}, df-cols:{df.columns.tolist()}, pp_text_dd:{pp_text_dd}, pp_target_dd:{pp_target_dd}')
    #df = df.copy()
    #df = df[[pp_text_dd, pp_target_dd]] # this will now work,local variable 'df' referenced before assignment 
    
    print('#1. filter df for text and target')
    df1 = df[[pp_text_dd, pp_target_dd]]
    print(f'df1-shp:{df1.shape}, df1-cols:{df1.columns.tolist()}')
    
    print('#2. rename columns as text and target')
    global target_col, text_col
    target_col = 'target'
    text_col='text'
    df1.rename(columns={pp_text_dd: text_col, pp_target_dd:target_col}, inplace=True)
    print(f'df1-shp:{df1.shape}, df1-cols:{df1.columns.tolist()}')
    
    
    print('#3. create project folder') #Oct01,2022
    eda_text_input_project = 'D:\\NLP1\\' # temp path_text to be added in FE
    print('eda_text_input_project:', eda_text_input_project)
    global data_path, pred_path, reports_path, model_path, NN_model_path, plot_path
    if eda_text_input_project:
        project_path = eda_text_input_project
    else: # create default AIML folder @ current working dir and add folders as per folder_list
        project_folder = os.getcwd()+'\\NLP1'

        if not os.path.exists(project_folder):
            os.mkdir(project_folder)
        
        project_path = project_folder+'\\'+target_col # Sep15,2022
        
    folder_list = ['data', 'pred', 'reports', 'models', 'NN_models', 'plots']
    data_path, pred_path, reports_path, model_path, NN_model_path, plot_path = create_ml_folders(project_path, folder_list)
    print('eda_text_input_project:', eda_text_input_project)
    print(f'data_path:{data_path}, pred_path:{pred_path}, reports_path:{reports_path}, model_path:{model_path},\
    NN_model_path:{NN_model_path}, plot_path:{plot_path}')
    
    
    print('#4. set default values')
    text_feat = 1 if pp_text_feat_chk is True else 0
    text_clean = 1 if pp_text_clean_chk is True else 0
    text_aug = 1 if pp_text_aug_chk is True else 0
    data_scale = 1 if pp_data_scale_chk is True else 0
    feat_select = 1 if pp_feat_select_chk is True else 0
    
    
    pp_ngram = pp_ngram_slider
    if pp_min_df is None: pp_min_df = 0.0
    if pp_max_df is None: pp_max_df = 1.0   
    if pp_max_feat is None: pp_max_feat = 100
    if pp_analyzer_dd is None : pp_analyzer_dd = 'word'
        
    selected_value = f'pp_text_dd:{pp_text_dd}, pp_target_dd:{pp_target_dd}, text_feat:{text_feat}, text_clean:{text_clean},\
            text_aug:{text_aug}, data_scale:{data_scale}, pp_text_encoding_radio:{pp_text_encoding_radio},\
            pp_min_df:{pp_min_df}, pp_max_df:{pp_max_df}, pp_ngram_slider:{pp_ngram}, pp_analyzer_dd:{pp_analyzer_dd}'
    print(f'df-shp:{df1.shape}, df-cols:{df1.columns.tolist()}, selected_value:{selected_value}')
    
    print('#5. text features')
    print(f'df-typ:{type(df1)}, df-cols:{df1.columns}')
    #text_col = 'text'
    if text_feat == 1:
        df1= extract_text_features(data=df1, STOPWORDS_list=STOPWORDS_list, stopword=True, fillna=True,
                          text_col=text_col, word_feat=True, sentiment=True, pos=True, drop_text_col=False)
        print(f'df1-shp:{df1.shape}, df1-cols:{df1.columns.tolist()}')
        
        # text feature to be created before text clean, to preserve features for X_train, X_valid,
        X=df1[text_col] # return series after split
        y=df1[target_col]
        X_train, X_valid, _, _ = train_test_split(X, y, random_state=0) # return series of X_train, X_valid w.r.t X,y
        print(f'X_train:{type(X_train)}, X_valid:{type(X_valid)}')
        X_train_feat_df = df1.iloc[X_train.index,:].drop([text_col, target_col], axis=1) # only text features with split index
        X_valid_feat_df = df1.iloc[X_valid.index,:].drop([text_col, target_col], axis=1) # only text features with split index
        
    
    print('#6. text clean')
    if text_clean == 1:
        df1 = text_cleaning(data=df1, STOPWORDS_list=STOPWORDS_list, text_col=text_col,
                n_rare_words = 10, n_most_common=10, series_out=False)
        print(f'df1-shp:{df1.shape}, df1-cols:{df1.columns.tolist()}')
        
    print('#7. text augmenation')
    if text_aug == 1:
        pass
    
    print('#8. data imbalance')
        
    print('#9. targt encoding / dict')
    target_key_set = set(df1[target_col].values)
    df1[target_col] = df1[target_col].factorize()[0]
    target_val_set = set(df1[target_col].values)
    n_target = len(target_key_set)
    target_dict = dict(zip(target_key_set, target_val_set))

    print('#10. separate X, y')
    X=df1[text_col].values # return arrays after split
    y=df1[target_col].values
    print(f'X-shp:{X.shape}, y-unq:{set(y)}, target_dict:{target_dict}, n_target:{n_target}')
    
    print('#11. split train/valid')
    from sklearn.model_selection import train_test_split
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=0) # return arrays of X_train, X_valid w.r.t X,y
          
    print('#12. X encoding / Embedding')
    object_param_dict = {}
    
    if (pp_text_encoding_radio == 'Count_vect') | (pp_text_encoding_radio == 'TfIdf_vect'):
        vect, X_train = text_BoW_feature_selection_train(X_train, y_train, analyzer=pp_analyzer_dd,
                max_df=pp_max_df, min_df=pp_min_df, max_features=pp_max_feat, ngram_range=pp_ngram, p_value_limit = 0.95,
                matrix_type=pp_data_type_radio, BoW_type=pp_text_encoding_radio, feature_selection=feat_select)

        X_valid = text_BoW_feature_selection_test(vect, X_valid, matrix_type=pp_data_type_radio)
        object_param_dict['vect'] = vect
        
        # merge text features , needed only for classical models
        if text_feat == 1:
            X_train = add_vect_text_features(vect, vect_matrix=X_train, text_feat_df=X_train_feat_df)
            X_valid = add_vect_text_features(vect, vect_matrix=X_valid, text_feat_df=X_valid_feat_df)

    elif pp_text_encoding_radio == 'Glove_Embedding':
        X_train, X_valid, vocab_size, tokenizer =  text_to_int(X_train, X_valid, padding_maxlen=100)
        object_param_dict['vocab_size'] = vocab_size
        object_param_dict['tokenizer'] = tokenizer
        
        # merge text features , not needed for NN models, But TBC how to merge with embedding
        if text_feat == 1:#TBA
            pass
        
    print('#13. Feature Scaling') # for scaling convert sparse to dense and after scaling convert back to sparse
    if data_scale == 1:
        from sklearn.preprocessing import MinMaxScaler
        import numpy as np
        scaler_X = MinMaxScaler(feature_range = (0, 1))
        #scaler_y = MinMaxScaler(feature_range = (0, 1))
    
#         #scaling works only on dense data
#         X_train = X_train.todense()
#         X_valid = X_valid.todense()

#         scaler_X.fit(X_train)
#         X_train = scaler_X.transform(X_train)
#         X_valid = scaler_X.transform(X_valid)
        
#         #convert back to sparse
#         from scipy import sparse
#         X_train=sparse.csr_matrix(X_train)
#         X_valid=sparse.csr_matrix(X_valid)

        #object_list.append[scaler_X] # other objects to be added
        object_param_dict['scaler_X'] = scaler_X

    print('#14. final test_type update')
    test_type = f'{project}--{pp_text_encoding_radio}-{pp_data_type_radio}-{text_feat}-{text_clean}-{text_aug}-{data_scale}-{feat_select}'
    print('test_type:', test_type)
    
    print('#15. update NLP objests eg. encoding/scaler')
    #object_list = [vect]
    FileName = 'NLP_objects_params'
    scaled_df = chk_create_update_df_dict(data=None, file_path=data_path, file_name=FileName,
                            dict_key=test_type, dict_value=object_param_dict, save_df=None)
    
    print('#16. update pre-processed data in pickle')
    FileName = 'NLP_train_valid'
    data_pp_Xy_tpl = X_train, X_valid, y_train, y_valid
    report_df = chk_create_update_df_dict(data=None, file_path=data_path, file_name=FileName,
                            dict_key=test_type, dict_value=data_pp_Xy_tpl, save_df=None)

    output_dispay = selected_value+'_'+test_type
    fig = make_subplots(specs=[[{"secondary_y": True}]]) # Create figure with secondary y-axis
    return fig, output_dispay
    
#################################Common#########################
#1 drop down PP data

@app.callback( 
    [Output('ml_pp_ptrn_dd', 'options'),
    Output('nn_pp_ptrn_dd', 'options')],
    Input('NLP_Classification', 'value'), prevent_initial_call=True) #dcc.Tabs(id="NLP_Classification", value='NLP Classification', children=[

def dropdown_pp_data(tab):
    print('callback dropdown_pp_data')
    FileName = 'NLP_train_valid'+'_pkl'
    if tab is None:
        raise PreventUpdate
    else:
        #if (tab == 'ML_Model'): #dcc.Tab(label='Stat Model', value='Stat_Model', children=[
        if (tab == 'ML_Model') or (tab == 'NN_Model'): #dcc.Tab(label='Stat Model', value='Stat_Model', children=[
            if os.path.exists(data_path+FileName):
                infile = open(data_path+FileName, 'rb')
                null_df = pickle.load(infile)
                data_pp_dict = pickle.load(infile)
            else:
                data_pp_dict = {'data_pp to be executed1': 2, 'data_pp to be executed2': 3}
        else:
            if os.path.exists(data_path+FileName):
                infile = open(data_path+FileName, 'rb')
                null_df = pickle.load(infile)
                data_pp_dict = pickle.load(infile)
            else:
                data_pp_dict = {'data_pp to be executed1': 2, 'data_pp to be executed2': 3}

        ml_options=[{'label':x, 'value':x} for x in list(data_pp_dict.keys())] # ML tabs
        data_pp_dict1 = {key:val for key, val in data_pp_dict.items() if 'Embedding' in key}
        nn_options=[{'label':x, 'value':x} for x in list(data_pp_dict1.keys())] # NN tabs , it doe snot work for CountVect/TfIdf
        
        #print('options:', options)
    #return options
    return [ml_options, nn_options]

#################################ML Models#########################
# tab : ML Model - callback function

#3 final callback
@app.callback(
    [
    Output('ML_model', 'figure'),
    Output("output_ml_model", "children"),
    ],
    [
     Input('ml_pp_ptrn_dd', 'value'), 
     Input('ml_models_chk', 'value'),
     Input('ml_score_dd', 'value'),
     Input('pp_training_type_radio', 'value'),
     Input("ml_btn", "n_clicks")
    ], prevent_initial_call=True)

def update_ML_model_final(ml_pp_ptrn_dd, ml_models_chk, ml_score_dd, pp_training_type_radio, ml_btn):
    print('callback update_ML_model_final')
    selection_status = f'selected ml_pp_ptrn_dd: {ml_pp_ptrn_dd}, ml_models_chk: {ml_models_chk},\
    ml_score_dd:{ml_score_dd}, pp_training_type_radio:{pp_training_type_radio}'
    print('selection_status:', selection_status)
    '''
    Known Error:
    to be done:
    feature importance:
    '''
    print('1. setup default values')
    if ml_score_dd is None: ml_score_dd = 'auc_prc_score'   
    
    print('2. read pre-processed data')
    FileName = 'NLP_train_valid'+'_pkl'
                              
    if os.path.exists(data_path+FileName):
        infile = open(data_path+FileName, 'rb')
        null_df = pickle.load(infile)
        data_pp_dict = pickle.load(infile)
    else:
        data_pp_dict = {'data_pp to be executed': 2}
        
    test_type = ml_pp_ptrn_dd
    X_train, X_valid, y_train, y_valid = data_pp_dict[ml_pp_ptrn_dd]
    print(f'X_train:{X_train.shape}')
                              
    print('3. read object list to update further')
    FileName = 'NLP_objects_params'+'_pkl'
                              
    if os.path.exists(data_path+FileName):
        infile = open(data_path+FileName, 'rb')
        null_df = pickle.load(infile)
        object_dict = pickle.load(infile)
    else:
        object_dict = {'data_pp to be executed': 2}
           
    #object_list = object_dict[ml_pp_ptrn_dd]
    object_param_dict = object_dict[ml_pp_ptrn_dd]
                              
    print('4. train the selected models')                         
    if ml_btn is None:
        raise PreventUpdate
    else:
        print('ml_models_chk:', ml_models_chk)
        for model_key in ml_models_chk:
            #model = models_dict[model_key]
            model = models_fdictC[model_key] # Oct01,2022
            model_name = model.__class__.__name__
            print('model in loop:', model_name)
            
            X=None; y=None; X_train=X_train; X_valid=X_valid; X_test=None; y_test=None; cv=3
            report_dict, y_preds, y_probs, y_actual, trained_model = tt_cv_pt(X, y, X_train, y_train,
                X_valid, y_valid, X_test, y_test,  model=model, param_grid=None, cv=cv, train_type=pp_training_type_radio)
            
            #temp
#             class_type =
#             class_mapping_dict =
#             target_col =
#             test_type =
#             report_dict, pred_df, y_probs, trained_model = tt_cv_pt_class(X, y, X_train, y_train, X_valid, y_valid,
#                         X_test=None, y_test=None, model=model, param_grid=None, n_splits=2, train_type=pp_training_type_radio,
#                         class_type = class_type, class_mapping_dict=class_mapping, target=target_col, data_table = True, test_type=test_type)
            
            # add score/error in test_type
            score_error = round(report_dict[ml_score_dd], 2)
            #test_type = ml_pp_ptrn_dd+'__ML-'+model_name+'__'+ml_score_dd+'-'+str(score_error)
            test_type = test_type+'-'+f'{pp_training_type_radio}__ML-{model_name}__{ml_score_dd}-{score_error}' # for ptrn dict_key
            report_dict['test_type'] = test_type # it will overwrite earlier test_type by new test_type with score
            print('test_type:', test_type)
            
            print('5. update models score in pickle')
            FileName = 'NLP_model_score'
            report_df = chk_create_update_df_dict(data=report_dict, file_path=data_path, file_name=FileName,
                                    dict_key=test_type, dict_value=trained_model, save_df=None)
                              
            print('6. update NLP objests eg. trained models')
            #object_list.append(trained_model)
            print('object_param_dict before update:', object_param_dict)
            object_param_dict['trained_model'] = trained_model
            print('object_param_dict after update:', object_param_dict)
                              
            FileName = 'NLP_objects_params'
            scaled_df = chk_create_update_df_dict(data=None, file_path=data_path, file_name=FileName,
                                    dict_key=test_type, dict_value=object_param_dict, save_df=None)
                              
    selection_status = f'selected ml_pp_ptrn_dd: {ml_pp_ptrn_dd}, ml_models_chk: {ml_models_chk}, ml_score_dd:{ml_score_dd},\
    score_dict:{report_dict}' 
    print('selection_status:', selection_status)
    
    output_dispay = selection_status+'_'+test_type
    fig = make_subplots(specs=[[{"secondary_y": True}]]) # Create figure with secondary y-axis
    return fig, output_dispay

#################################NN Models#########################
#NN Models
@app.callback(
    [
    Output('NN_model', 'figure'),
    Output("output_nn_model", "children"),
    ],
    [
     Input('nn_pp_ptrn_dd', 'value'), 
     Input('nn_models_chk', 'value'),    
     Input('nn_cols_chk', 'value'),
     Input('n_epochs', 'value'),
     Input('n_neurons', 'value'),
     Input('n_batch', 'value'),
     Input('n_repeats', 'value'),
     Input('nn_lr_slider', 'value'),
     Input('nn_optimizer_dd', 'value'),
     Input('nn_score_dd', 'value'),
     Input('nn_loss_dd', 'value'),
     Input("nn_btn", "n_clicks")
    ],prevent_initial_call=True)

def update_NN_model_final(nn_pp_ptrn_dd, nn_models_chk, nn_cols_chk, n_epochs, n_neurons, n_batch, n_repeats, nn_lr_slider,
                    nn_optimizer_dd, nn_score_dd, nn_loss_dd, nn_btn):
    print('callback update_NN_model_final')
    
    '''
    Known Error:
    1. score/loss does not support 'mean'
    
    to be done:
    '''
    print('# setup default values')
    if n_epochs is None: n_epochs = 2
    if n_neurons is None: n_neurons = 20
    if n_batch is None: n_batch = 2
    
    if nn_optimizer_dd is None: nn_optimizer_dd = 'Adam'
    if nn_score_dd is None: nn_score_dd = 'mean_absolute_error'
    if nn_loss_dd is None: nn_loss_dd = 'mean_absolute_error' 
    #print(f'default values->nn_forecast_period:{nn_forecast_period}, nn_steps_in_window:{nn_steps_in_window}')

    selection_status = f'selected nn_pp_ptrn_dd: {nn_pp_ptrn_dd}, nn_models_chk: {nn_models_chk}, nn_cols_chk: {nn_cols_chk}' #add other i/p
    print('selection_status:', selection_status)
     
    print('2. read pre-processed data')
    FileName = 'NLP_train_valid'+'_pkl'
                              
    if os.path.exists(data_path+FileName):
        infile = open(data_path+FileName, 'rb')
        null_df = pickle.load(infile)
        data_pp_dict = pickle.load(infile)
    else:
        data_pp_dict = {'data_pp to be executed': 2}
        
    test_type = nn_pp_ptrn_dd
    X_train, X_valid, y_train, y_valid = data_pp_dict[nn_pp_ptrn_dd]
    print(f'X_train:{X_train.shape}')
                              
    print('3. read object list to update further')
    FileName = 'NLP_objects_params'+'_pkl'
                              
    if os.path.exists(data_path+FileName):
        infile = open(data_path+FileName, 'rb')
        null_df = pickle.load(infile)
        object_dict = pickle.load(infile)
    else:
        object_dict = {'data_pp to be executed': 2}
           
    #object_list = object_dict[ml_pp_ptrn_dd]
    object_param_dict = object_dict[nn_pp_ptrn_dd]
    
    if (nn_pp_ptrn_dd.find('Embedding') != -1):
        vocab_size = object_param_dict['vocab_size']
        tokenizer = object_param_dict['tokenizer']
                              
    print('4. train the selected models')                         
    if nn_btn is None:
        raise PreventUpdate
    else:
        for model in nn_models_chk:
            print('model in loop:', model)
            model_name = model
            '''
            pending:
            num_classes / retrun-model_eval_dict TBC
            '''
            
            if model.startswith('lrng'):
                vector_size = 300 #TBC X_train.shape[0]
                trained_weight=None
            elif model.startswith('trnd'):
                glove_embedding_vector = load_Glove_embedding_vector(glove_path+glove_file)
                embedding_matrix = embedding_vector_2_matrix(vocab_size, tokenizer, glove_embedding_vector)

                vector_size = len(list(glove_embedding_vector.values())[0]) # find vector size
                trained_weight = embedding_matrix
                
            model_return = build_NN_text_class_model(vocab_size, vector_size, X_train, y_train, X_valid, y_valid,
                                       trained_weight, model_type=model, num_classes = 1,
                                       score_metrics = nn_score_dd, DropOutRate=0.25, optimizer = nn_optimizer_dd,
                                       n_epochs = n_epochs, batch_size=n_batch, n_neurons= n_neurons, learning_rate=nn_lr_slider,
                                       l2_loss_lambda=0.10, model_save_path='D:\\NLP\\models\\NN_models', test_type=test_type)

            #trianed_model, history_df, train_loss, train_acc, valid_loss, valid_acc = model_return
            model_eval_dict, test_pred, trianed_model, history_df = model_return
            print('model_eval_dict:', model_eval_dict)
            val_loss1 = round(history_df['val_loss'].iloc[-1], 2) #TBC which is best value

            #test_type = f'{nn_pp_ptrn_dd}__step_in-{nn_steps_in_window}__NN-{model}__val_loss-{val_loss1}'
            test_type = f'{nn_pp_ptrn_dd}__NN-{model}__val_loss-{val_loss1}'
            print(f'test_type:{test_type}')
            NN_model_save_path_name = NN_model_path+'\\'+test_type+'.h5'
            trianed_model.save(NN_model_save_path_name)

#             preds_metrics_dict = classification_error_score_metrics(y_actual=y_valid, y_preds=test_pred, y_probs=None,
#                             model_name=model, test_type=test_type, n_features=1, plot=False, plot_path=None)
#             print('preds_metrics_dict:', preds_metrics_dict)


#             test_type = f'model_eval_dict: {model_eval_dict}, optimizer:{nn_optimizer_dd}, price_slid:{nn_price_slider},\
#             lrearning_rate:{nn_lr_slider}, n_epochs:{n_epochs}, n_batch:{n_batch}, n_neurons:{n_neurons}'
#             print(f'history_df-shp:{history_df.shape}, history_df-cols:{history_df.columns}')

#             print('\n #6 regression other metrics')
#             print(f'valid_y-len:{len(valid_y)}, test_pred-len:{len(test_pred)}')
#             report_dict = regression_error_score_metrics(actual_y=valid_y, preds=test_pred, model_name=model,
#                                                          n_features=n_features, plot=False, plot_path=None)
#             #report_dict['n_repeats'] = i
            
#             # add selected_cols_list/all_cols_list in report_dict
#             report_dict['selected_cols_list'] =  ','.join(selected_cols_list)
#             report_dict['all_cols_list'] =  ','.join(all_cols_list)
            
#             report_dict['train_loss'] = history_df['loss'].iloc[-1]
#             report_dict['val_loss'] = history_df['val_loss'].iloc[-1]

#             #report_dict['forecast_period'] = nn_forecast_period
#             #report_dict['n_records'] = nn_n_records
#             report_dict['price_range'] = nn_price_slider

#             report_dict['n_features'] = n_features
#             report_dict['nn_steps_out'] = nn_forecast_period
#             report_dict['nn_steps_in'] = nn_steps_in_window

#             report_dict['optimizer'] = nn_optimizer_dd
#             report_dict['loss'] = nn_loss_dd
#             report_dict['score'] = nn_score_dd

#             report_dict['n_epochs'] = n_epochs
#             report_dict['n_batch'] = n_batch
#             report_dict['n_neurons'] = n_neurons
#             report_dict['learning_rate'] = nn_lr_slider
#             report_dict['test_type'] = test_type

#             #save report_dict
#             FileName = 'dash_report_df_Score'
#             report_df = chk_create_update_df_dict(data=report_dict, file_path=data_path, file_name=FileName,
#                                                   dict_key=test_type, dict_value=None, save_df=df_tail)
#             print('#save scaler_X, scaler_y')
#             FileName = 'ts_scaled_Xy'
#             scaled_df = chk_create_update_df_dict(data=None, file_path=data_path, file_name=FileName,
#                                                   dict_key=test_type, dict_value=scaler_Xy, save_df=None)

            print('#out data')
            out_data_nn = f'optimizer:{nn_optimizer_dd}, lrearning_rate:{nn_lr_slider}, n_epochs:{n_epochs}, n_batch:{n_batch},\
            n_neurons:{n_neurons}, test_type:{test_type}'
            print(f'history_df-shp:{history_df.shape}, history_df-cols:{history_df.columns}')

    fig_nn = make_subplots(rows=1, cols=2)
    fig_nn.append_trace(go.Scatter(x=history_df.index, y=history_df['loss'], name='loss', line = dict(color='royalblue', width=4, dash='dash')), row=1, col=1)
    fig_nn.append_trace(go.Scatter(x=history_df.index, y=history_df['val_loss'], name='val_loss', line=dict(color='red', width=4, dash='dot')), row=1, col=1)
    fig_nn.append_trace(go.Scatter(x=history_df.index, y=history_df[nn_score_dd], name=nn_score_dd, line = dict(color='blue', width=4, dash='dash')), row=1, col=2)
    fig_nn.append_trace(go.Scatter(x=history_df.index, y=history_df['val_'+nn_score_dd], name='val_'+nn_score_dd, line=dict(color='yellow', width=4, dash='dot')), row=1, col=2)

    return fig_nn, out_data_nn
    
# if __name__ == '__main__':
#     app.run_server(port=8055)

In [ ]:
if __name__ == '__main__':
    app.run_server(port=8058)

Dash is running on http://127.0.0.1:8058/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8058/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Nov/2022 11:23:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2022 11:23:59] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2022 11:23:59] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2022 11:23:59] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2022 11:23:59] "POST /_dash-update-component HTTP/1.1" 200 -


callback data_pp_dd_text_target1
read_file: None
callback show_hide_element_Data_pp2
else other
Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\ajverma\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 2077, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\ajverma\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 1525, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\ajverma\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 1523, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\ajverma\AppData\Roaming\Python\Python39\site-packages\flask\app.py", line 1509, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**req.view_args)
  File "C:\ProgramData\Anaconda3\lib\site-packages\dash\dash.py", line 1336, in dispatch
    response.set_data(func(*args, outputs_list=outputs_list))
  File 

127.0.0.1 - - [18/Nov/2022 11:23:59] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [18/Nov/2022 11:24:01] "GET /_dash-component-suites/dash/dcc/async-upload.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2022 11:24:01] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2022 11:24:01] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2022 11:24:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2022 11:24:01] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Nov/2022 11:24:01] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -


callback dropdown_pp_data


...

callback show_hide_element_Data_pp2
else other


127.0.0.1 - - [18/Nov/2022 11:26:33] "POST /_dash-update-component HTTP/1.1" 200 -


callback show_hide_element_Data_pp2
else other


127.0.0.1 - - [18/Nov/2022 12:01:46] "POST /_dash-update-component HTTP/1.1" 200 -


callback show_hide_element_Data_pp2
else other
